In [160]:
### Download renewables in new buildings data for Landkreise
### from inkar.de and compile into csv with Regions as rows
### and years as columns.

In [161]:
#https://www.regionalstatistik.de/genesis/online?operation=previous&levelindex=3&step=2&titel=Tabellenaufbau&levelid=1655207886087&levelid=1655207857464#abreadcrumb
#Fertigstellungen neuer Wohngebäude und Nichtwohngebäude sowie Wohnungen in Wohngebäuden nach Zahl der Wohnungen und vorwiegender Art der Beheizung - Jahressumme
# downloaded as "flat"

In [162]:
# import necessary libraries
import numpy as np
import pandas as pd

In [163]:
## Dataset: regionalstatistik

#helper function to move values around 
#needed because of structural problems with the raw data
def move_to_third_section(df,title,value):
    second_section = "2_" + title;
    third_section = "3_" + title;

    if "Merkmal" in title:
        df[third_section] = np.where((df[second_section] == value), df[second_section],df[third_section])
        #df[second_section] = np.where((df[second_section] == value),"WOHNGEBAUDE GESAMT",df[third_section])

    elif "Auspraegung" in title:
        merkmal = second_section.replace("Auspraegung","Merkmal")
        df[third_section] = np.where((df[merkmal] == value), df[second_section],df[third_section])
        #df[second_section] = np.where((df[merkmal] == value),"WOHNGEBAUDE GESAMT",df[third_section])

    else:
        print("unknown title: ", title)

    return df

In [164]:
#preprocess dataset
def preprocess_dataset_kreise(df):

    #set AGS Germany to 0 (so we only have numvers)
    df["1_Auspraegung_Code"] = df["1_Auspraegung_Code"].replace({"DG":"0"})    
    df["1_Auspraegung_Code"] = pd.to_numeric(df["1_Auspraegung_Code"])

    #clean up multi level format of raw data 
    df["2_Auspraegung_Label"] = df["2_Auspraegung_Label"].replace({"Insgesamt":'Wohnungen Insgesamt'})  
    #df["2_Auspraegung_Label"] = df["2_Auspraegung_Label"].str.replace('Insgesamt','Wohnungen Insgesamt',regex=True)
    df.loc[df["2_Auspraegung_Code"] == "INSGESAMT","2_Auspraegung_Label"] = "Wohngebäude Insgesamt"

    #rename titles
    titles = ["Merkmal_Code","Merkmal_Label","Auspraegung_Code","Auspraegung_Label"]

    for title in titles:
        if "Code" in title:
            df = move_to_third_section(df,title,"BAUPHE")
        elif "Label" in title:
            df = move_to_third_section(df,title,"Primär verwendete Heizenergie")

    for title in reversed(titles):
        title = "2_" + title
        df[title] = np.where((df["2_Merkmal_Code"] == "BAUPHE"),"Wohngebäude Insgesamt",df[title])

    #rename columns
    df.columns = df.columns.str.replace("1_", "Ort_")
    df.columns = df.columns.str.replace("2_", "Wohnort_")
    df.columns = df.columns.str.replace("3_", "Heizenergie_")


    #make building counts numeric
    building_counts = ["BAU017__Wohngebaeude_insgesamt__Anzahl","WOHN04__Wohnungen_in_Wohngebaeuden_(ohne_Wohnheime)__Anzahl","BAU018__Nichtwohngebaeude_insgesamt__Anzahl"]
    
    for building in building_counts:
        df[building] = df[building].str.replace('-','0',regex=True)
        df[building].fillna(0,inplace=True)
        df[building] = pd.to_numeric(df[building])


    #set total number of buildings for each year and place
    #needed to calculate proportions
    years = list(range(2016, 2021))
    orte = df["Ort_Auspraegung_Code"].unique()

    for ort in orte:
        for year in years:
            isLocation = (df["Ort_Auspraegung_Code"] == ort)
            isYear = (df["Zeit"] == year)

            gesamt = df.loc[isLocation & isYear & (df["Heizenergie_Auspraegung_Code"] == "INSGESAMT")].copy()
            
            gesamt_wohngebaeude = gesamt.loc[(gesamt["Wohnort_Auspraegung_Label"] == "Wohngebäude Insgesamt"),"BAU017__Wohngebaeude_insgesamt__Anzahl"]
            gesamt_wohnungen = gesamt.loc[(gesamt["Wohnort_Auspraegung_Label"] == "Wohnungen Insgesamt"),"WOHN04__Wohnungen_in_Wohngebaeuden_(ohne_Wohnheime)__Anzahl"]

            if(gesamt_wohngebaeude.size == 1 and gesamt_wohnungen.size == 1):
                df.loc[isLocation & isYear,"Wohngebäude_Gesamt"] = int(gesamt_wohngebaeude.item())
                df.loc[isLocation & isYear,"Wohnungen_Gesamt"] = int(gesamt_wohnungen.item())
            else:
                print("error: gesamt is wrong",gesamt)

    #calculate proportions
    df["Wohngebäude_Anteil"] = df["BAU017__Wohngebaeude_insgesamt__Anzahl"] / df["Wohngebäude_Gesamt"] * 100
    df["Wohnungen_Anteil"] = df["WOHN04__Wohnungen_in_Wohngebaeuden_(ohne_Wohnheime)__Anzahl"] / df["Wohnungen_Gesamt"] * 100

    #Drop unneccessary columns to save space
    df.drop(['Statistik_Label','Ort_Merkmal_Label'], axis=1, inplace = True)

    return df

In [165]:
#create pivoted data table
#         viz : df dataframe slice of your data, for example a specific landkreis
#        asProportion : bool, optional
#            if true, the visualisation will use proportion. otherwise, the total number is used. default is true
#        useWohnungen : bool, optional
#            if true, flats will be counted. otherwise, buildings will be counted (which maybe have multiple flats in them). default is true
def create_pivot_tables(df_p,asProportion = True,useWohnungen = True):

    df = df_p.copy()
    
    #rename columns
    df.rename({"Zeit": "year","BAU017__Wohngebaeude_insgesamt__Anzahl":"buildings","Heizenergie_Auspraegung_Label":"energy",
        "Wohngebäude_Anteil":"buildings_anteil","Wohnungen_Anteil":"flats_anteil",
        "WOHN04__Wohnungen_in_Wohngebaeuden_(ohne_Wohnheime)__Anzahl":"flats"}, axis=1,inplace=True)

    # dont add gesamtenergie to plot
    df = df[df["energy"] != "Wohngebäude Insgesamt"]
    df = df[df["energy"] != "Wohnungen Insgesamt"]
    df = df[df["energy"] != "Insgesamt"]

    if useWohnungen:
        column="flats"
        
        #for wohnungen: do not differ between 1, 2 or 3 or more flats
        df= df[df["Wohnort_Auspraegung_Label"] == "Wohnungen Insgesamt"]
    
    else:
        column='buildings'
        df= df[df["Wohnort_Auspraegung_Label"] == "Wohngebäude Insgesamt"]
    
    if asProportion:
        column += "_anteil"

    #pivot data
    orte = df["Ort_Auspraegung_Code"].unique()

    pivot_total = pd.DataFrame()

    for ort in orte:
        snippet = df[df["Ort_Auspraegung_Code"] == ort]

        temp = pd.pivot_table(snippet, index='year',columns='energy', values=column)
        temp = temp.reset_index(level=0)

        temp["ort_ags"] = ort
        temp["ort_name"]= str(snippet["Ort_Auspraegung_Label"].iloc[0])

        pivot_total = pd.concat([pivot_total, temp])

    #sum up renewables and fossils
    renewables = ["Solarthermie","Geothermie","Umweltthermie (Luft/Wasser)","Holz","Biogas/Biomethan","Sonstige Biomasse","Keine Energie (einschl. Passivhaus)"]
    fossils = ["Gas","Strom","Öl","Fernwärme/Fernkälte","Sonstige Heizenergie"]

    pivot_total["renewables"] = pivot_total[renewables].sum(axis=1)
    pivot_total["fossils"] = pivot_total[fossils].sum(axis=1)


    
    pivot_total.to_csv("data/newbuildings_pivot_"+column+".csv")
    return pivot_total

In [166]:
def final_data_transform(pivot):
    #change year column to string to facilitate renaming columns after second pivot
    pivot['year'] = pivot['year'].astype(str)

    #pivot dataframe to have both energy types and years as columns
    piv_df = pivot.pivot(index=['ort_ags','ort_name'],columns='year',values=["renewables","fossils","Solarthermie","Geothermie","Umweltthermie (Luft/Wasser)","Holz","Biogas/Biomethan","Sonstige Biomasse","Keine Energie (einschl. Passivhaus)","Gas","Strom","Öl","Fernwärme/Fernkälte","Sonstige Heizenergie"])

    #collapse levels of column names and join energy type to year
    piv_df.columns = piv_df.columns.map('_'.join)

    #prepend all columns with indicator name
    piv_df=piv_df.add_prefix('buildings_newbuilding_energy_')

    #reset index to recreate AGS and Name columns
    piv_df.reset_index(inplace=True)

    #rename Name and AGS columns and strip whitespace
    piv_df.rename(columns={'ort_name':'Name','ort_ags':'AGS'},inplace=True)

    piv_df['Name']=piv_df['Name'].str.strip()

    #set AGS as index
    piv_df.set_index('AGS',inplace=True)

    #add column for human-readable unit of measure
    piv_df['Unit'] = "fertiggestellte Wohngebäude"

    return piv_df

In [167]:
#load dataset from local file
nb_kreise = pd.read_csv("data/newbuildings_kreise.csv",delimiter=";", encoding='latin1', low_memory=False)

#preprocessing
nb_kreise = preprocess_dataset_kreise(nb_kreise)
nb_kreise

,Statistik_Code,Zeit_Code,Zeit_Label,Zeit,Ort_Merkmal_Code,Ort_Auspraegung_Code,Ort_Auspraegung_Label,Wohnort_Merkmal_Code,Wohnort_Merkmal_Label,Wohnort_Auspraegung_Code,...,Heizenergie_Merkmal_Label,Heizenergie_Auspraegung_Code,Heizenergie_Auspraegung_Label,BAU017__Wohngebaeude_insgesamt__Anzahl,WOHN04__Wohnungen_in_Wohngebaeuden_(ohne_Wohnheime)__Anzahl,BAU018__Nichtwohngebaeude_insgesamt__Anzahl,Wohngebäude_Gesamt,Wohnungen_Gesamt,Wohngebäude_Anteil,Wohnungen_Anteil
0,31121,JAHR,Jahr,2020,KREISE,0,Deutschland,Wohngebäude Insgesamt,Wohngebäude Insgesamt,Wohngebäude Insgesamt,...,Primär verwendete Heizenergie,BAUPHE00,Keine Energie (einschl. Passivhaus),111,0,12960,112935.0,261124.0,0.098287,0.000000
1,31121,JAHR,Jahr,2020,KREISE,0,Deutschland,Wohngebäude Insgesamt,Wohngebäude Insgesamt,Wohngebäude Insgesamt,...,Primär verwendete Heizenergie,BAUPHE02,Öl,776,0,272,112935.0,261124.0,0.687121,0.000000
2,31121,JAHR,Jahr,2020,KREISE,0,Deutschland,Wohngebäude Insgesamt,Wohngebäude Insgesamt,Wohngebäude Insgesamt,...,Primär verwendete Heizenergie,BAUPHE03,Gas,44037,0,5014,112935.0,261124.0,38.993226,0.000000
3,31121,JAHR,Jahr,2020,KREISE,0,Deutschland,Wohngebäude Insgesamt,Wohngebäude Insgesamt,Wohngebäude Insgesamt,...,Primär verwendete Heizenergie,BAUPHE04,Strom,1606,0,780,112935.0,261124.0,1.422057,0.000000
4,31121,JAHR,Jahr,2020,KREISE,0,Deutschland,Wohngebäude Insgesamt,Wohngebäude Insgesamt,Wohngebäude Insgesamt,...,Primär verwendete Heizenergie,BAUPHE05,Fernwärme/Fernkälte,9053,0,1648,112935.0,261124.0,8.016115,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174845,31121,JAHR,Jahr,2016,KREISE,16077,"Altenburger Land, Kreis",GEBWH1,Gebäude nach Anzahl der Wohnungen,WHGZHL03UM,...,Primär verwendete Heizenergie,BAUPHE09,Holz,0,0,0,58.0,101.0,0.000000,0.000000
174846,31121,JAHR,Jahr,2016,KREISE,16077,"Altenburger Land, Kreis",GEBWH1,Gebäude nach Anzahl der Wohnungen,WHGZHL03UM,...,Primär verwendete Heizenergie,BAUPHE10,Biogas/Biomethan,0,0,0,58.0,101.0,0.000000,0.000000
174847,31121,JAHR,Jahr,2016,KREISE,16077,"Altenburger Land, Kreis",GEBWH1,Gebäude nach Anzahl der Wohnungen,WHGZHL03UM,...,Primär verwendete Heizenergie,BAUPHE11,Sonstige Biomasse,0,0,0,58.0,101.0,0.000000,0.000000
174848,31121,JAHR,Jahr,2016,KREISE,16077,"Altenburger Land, Kreis",GEBWH1,Gebäude nach Anzahl der Wohnungen,WHGZHL03UM,...,Primär verwendete Heizenergie,BAUPHE12,Sonstige Heizenergie,0,0,0,58.0,101.0,0.000000,0.000000


In [168]:
# create_pivot_tables_
#         viz : df dataframe slice of your data, for example a specific landkreis
#        asProportion : bool, optional
#            if true, the visualisation will use proportion. otherwise, the total number is used. default is true
#        useWohnungen : bool, optional
#            if true, flats will be counted. otherwise, buildings will be counted (which maybe have multiple flats in them). default is true
pivot = create_pivot_tables(nb_kreise,True,True)
pivot

energy,year,Biogas/Biomethan,Fernwärme/Fernkälte,Gas,Geothermie,Holz,Keine Energie (einschl. Passivhaus),Solarthermie,Sonstige Biomasse,Sonstige Heizenergie,Strom,Umweltthermie (Luft/Wasser),Öl,ort_ags,ort_name,renewables,fossils
0,2016,0.414990,19.123869,49.870626,5.717292,5.278820,0.055543,0.494014,0.512529,0.411378,0.591553,16.798298,0.731087,0,Deutschland,29.271487,70.728513
1,2017,0.448011,19.228634,46.271455,5.482344,5.773879,0.058132,0.492594,0.503521,0.358846,0.543295,20.113729,0.725559,0,Deutschland,32.872210,67.127790
2,2018,0.290789,21.294326,41.498508,5.846834,6.000411,0.080565,0.624378,0.347856,0.707461,0.658367,22.029901,0.620602,0,Deutschland,35.220735,64.779265
3,2019,0.391204,22.645564,40.049073,6.003339,5.103123,0.070685,0.470014,0.348143,0.549230,0.747878,23.011582,0.610165,0,Deutschland,35.398090,64.601910
4,2020,0.422788,23.097073,37.808091,5.885327,4.605475,0.049402,0.412448,0.415128,0.398661,0.928295,25.459552,0.517762,0,Deutschland,37.250119,62.749881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2016,0.000000,2.970297,76.237624,5.940594,0.000000,0.000000,0.990099,0.990099,0.000000,0.000000,12.871287,0.000000,16077,"Altenburger Land, Kreis",20.792079,79.207921
1,2017,0.000000,4.411765,45.588235,8.823529,5.882353,0.000000,2.941176,1.470588,0.000000,0.000000,30.882353,0.000000,16077,"Altenburger Land, Kreis",50.000000,50.000000
2,2018,0.000000,6.944444,36.111111,12.500000,2.777778,0.000000,0.000000,0.000000,0.000000,8.333333,33.333333,0.000000,16077,"Altenburger Land, Kreis",48.611111,51.388889
3,2019,0.000000,69.620253,15.189873,1.898734,0.632911,0.000000,0.632911,0.000000,0.000000,0.632911,10.759494,0.632911,16077,"Altenburger Land, Kreis",13.924051,86.075949


In [169]:
piv_df = final_data_transform(pivot)
piv_df

,Name,buildings_newbuilding_energy_renewables_2016,buildings_newbuilding_energy_renewables_2017,buildings_newbuilding_energy_renewables_2018,buildings_newbuilding_energy_renewables_2019,buildings_newbuilding_energy_renewables_2020,buildings_newbuilding_energy_fossils_2016,buildings_newbuilding_energy_fossils_2017,buildings_newbuilding_energy_fossils_2018,buildings_newbuilding_energy_fossils_2019,...,buildings_newbuilding_energy_Fernwärme/Fernkälte_2017,buildings_newbuilding_energy_Fernwärme/Fernkälte_2018,buildings_newbuilding_energy_Fernwärme/Fernkälte_2019,buildings_newbuilding_energy_Fernwärme/Fernkälte_2020,buildings_newbuilding_energy_Sonstige Heizenergie_2016,buildings_newbuilding_energy_Sonstige Heizenergie_2017,buildings_newbuilding_energy_Sonstige Heizenergie_2018,buildings_newbuilding_energy_Sonstige Heizenergie_2019,buildings_newbuilding_energy_Sonstige Heizenergie_2020,Unit
AGS,,,,,,,,,,,,,,,,,,,,,
0,Deutschland,29.271487,32.872210,35.220735,35.398090,37.250119,70.728513,67.127790,64.779265,64.601910,...,19.228634,21.294326,22.645564,23.097073,0.411378,0.358846,0.707461,0.549230,0.398661,fertiggestellte Wohngebäude
1,Schleswig-Holstein,17.315705,24.738024,23.225152,23.774490,26.782061,82.684295,75.261976,76.774848,76.225510,...,16.598054,20.200996,23.501695,24.020456,0.905449,0.233907,1.705698,1.157312,1.164437,fertiggestellte Wohngebäude
2,Hamburg,7.976157,8.242170,9.143216,7.250221,5.865467,92.023843,91.757830,90.856784,92.749779,...,38.138768,40.433100,41.136163,55.257870,1.362475,1.198861,4.425149,5.526083,3.339393,fertiggestellte Wohngebäude
3,Niedersachsen,15.535673,18.121197,21.380828,20.966418,20.774876,84.464327,81.878803,78.619172,79.033582,...,6.693340,6.032300,7.654717,9.194906,0.507398,0.275069,0.514801,0.758435,0.259011,fertiggestellte Wohngebäude
4,Bremen,4.927353,8.592058,2.326870,7.053717,6.030856,95.072647,91.407942,97.673130,92.946283,...,28.447653,30.138504,37.601736,40.603086,0.126342,0.144404,0.886427,1.519262,0.841515,fertiggestellte Wohngebäude
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11008008,Berlin-Neukölln,NaN,NaN,5.533063,3.609626,3.000000,NaN,NaN,94.466937,96.390374,...,NaN,37.651822,74.866310,90.400000,NaN,NaN,0.000000,0.000000,0.000000,fertiggestellte Wohngebäude
11009009,Berlin-Treptow-Köpenick,NaN,NaN,5.057471,6.794294,17.111995,NaN,NaN,94.942529,93.205706,...,NaN,52.321839,66.028529,44.229676,NaN,NaN,0.000000,3.003003,0.284252,fertiggestellte Wohngebäude
11010010,Berlin-Marzahn-Hellersdorf,NaN,NaN,15.795455,6.398349,9.476584,NaN,NaN,84.204545,93.601651,...,NaN,45.795455,65.428277,81.157025,NaN,NaN,0.340909,0.000000,0.000000,fertiggestellte Wohngebäude


In [170]:
#change year column to string to facilitate renaming columns after second pivot
pivot['year'] = pivot['year'].astype(str)

#pivot dataframe to have both energy types and years as columns
piv_df = pivot.pivot(index=['ort_ags','ort_name'],columns='year',values=["renewables","fossils","Solarthermie","Geothermie","Umweltthermie (Luft/Wasser)","Holz","Biogas/Biomethan","Sonstige Biomasse","Keine Energie (einschl. Passivhaus)","Gas","Strom","Öl","Fernwärme/Fernkälte","Sonstige Heizenergie"])
piv_df

renewables                        \
year                                            2016       2017       2018   
ort_ags  ort_name                                                            
0        Deutschland                       29.271487  32.872210  35.220735   
1          Schleswig-Holstein              17.315705  24.738024  23.225152   
2          Hamburg                          7.976157   8.242170   9.143216   
3          Niedersachsen                   15.535673  18.121197  21.380828   
4          Bremen                           4.927353   8.592058   2.326870   
...                                              ...        ...        ...   
11008008       Berlin-Neukölln                   NaN        NaN   5.533063   
11009009       Berlin-Treptow-Köpenick           NaN        NaN   5.057471   
11010010       Berlin-Marzahn-Hellersdorf        NaN        NaN  15.795455   
11011011       Berlin-Lichtenberg                NaN        NaN   3.182086   
11012012       Berlin-Reinickendorf              NaN        NaN  28.244275   

                                                                   fossils  \
year                                            2019       2020       2016   
ort_ags  ort_name                                                            
0        Deutschland                       35.398090  37.250119  70.728513   
1          Schleswig-Holstein              23.774490  26.782061  82.684295   
2          Hamburg                          7.250221   5.865467  92.023843   
3          Niedersachsen                   20.966418  20.774876  84.464327   
4          Bremen                           7.053717   6.030856  95.072647   
...                                              ...        ...        ...   
11008008       Berlin-Neukölln              3.609626   3.000000        NaN   
11009009       Berlin-Treptow-Köpenick      6.794294  17.111995        NaN   
11010010       Berlin-Marzahn-Hellersdorf   6.398349   9.476584        NaN   
11011011       Berlin-Lichtenberg           2.387562   2.443992        NaN   
11012012       Berlin-Reinickendorf        19.672131   8.843537        NaN   

                                                                            \
year                                            2017       2018       2019   
ort_ags  ort_name                                                            
0        Deutschland                       67.127790  64.779265  64.601910   
1          Schleswig-Holstein              75.261976  76.774848  76.225510   
2          Hamburg                         91.757830  90.856784  92.749779   
3          Niedersachsen                   81.878803  78.619172  79.033582   
4          Bremen                          91.407942  97.673130  92.946283   
...                                              ...        ...        ...   
11008008       Berlin-Neukölln                   NaN  94.466937  96.390374   
11009009       Berlin-Treptow-Köpenick           NaN  94.942529  93.205706   
11010010       Berlin-Marzahn-Hellersdorf        NaN  84.204545  93.601651   
11011011       Berlin-Lichtenberg                NaN  96.817914  97.612438   
11012012       Berlin-Reinickendorf              NaN  71.755725  80.327869   

                                                      ... Fernwärme/Fernkälte  \
year                                            2020  ...                2016   
ort_ags  ort_name                                     ...                       
0        Deutschland                       62.749881  ...           19.123869   
1          Schleswig-Holstein              73.217939  ...           19.447115   
2          Hamburg                         94.134533  ...           50.085155   
3          Niedersachsen                   79.225124  ...            5.527152   
4          Bremen                          93.969144  ...           38.092230   
...                                              ...  ...                 ...   
11008008       Berlin-Neukölln    

In [171]:
#collapse levels of column names and join energy type to year
piv_df.columns = piv_df.columns.map('_'.join)

#prepend all columns with indicator name
piv_df=piv_df.add_prefix('buildings_newbuilding_energy_')

#reset index to recreate AGS and Name columns
piv_df.reset_index(inplace=True)

#rename Name and AGS columns and strip whitespace
piv_df.rename(columns={'ort_name':'Name','ort_ags':'AGS'},inplace=True)

piv_df['Name']=piv_df['Name'].str.strip()

#set AGS as index
piv_df.set_index('AGS',inplace=True)

#add column for human-readable unit of measure
piv_df['Unit'] = "fertiggestellte Wohngebäude"

piv_df

,Name,buildings_newbuilding_energy_renewables_2016,buildings_newbuilding_energy_renewables_2017,buildings_newbuilding_energy_renewables_2018,buildings_newbuilding_energy_renewables_2019,buildings_newbuilding_energy_renewables_2020,buildings_newbuilding_energy_fossils_2016,buildings_newbuilding_energy_fossils_2017,buildings_newbuilding_energy_fossils_2018,buildings_newbuilding_energy_fossils_2019,...,buildings_newbuilding_energy_Fernwärme/Fernkälte_2017,buildings_newbuilding_energy_Fernwärme/Fernkälte_2018,buildings_newbuilding_energy_Fernwärme/Fernkälte_2019,buildings_newbuilding_energy_Fernwärme/Fernkälte_2020,buildings_newbuilding_energy_Sonstige Heizenergie_2016,buildings_newbuilding_energy_Sonstige Heizenergie_2017,buildings_newbuilding_energy_Sonstige Heizenergie_2018,buildings_newbuilding_energy_Sonstige Heizenergie_2019,buildings_newbuilding_energy_Sonstige Heizenergie_2020,Unit
AGS,,,,,,,,,,,,,,,,,,,,,
0,Deutschland,29.271487,32.872210,35.220735,35.398090,37.250119,70.728513,67.127790,64.779265,64.601910,...,19.228634,21.294326,22.645564,23.097073,0.411378,0.358846,0.707461,0.549230,0.398661,fertiggestellte Wohngebäude
1,Schleswig-Holstein,17.315705,24.738024,23.225152,23.774490,26.782061,82.684295,75.261976,76.774848,76.225510,...,16.598054,20.200996,23.501695,24.020456,0.905449,0.233907,1.705698,1.157312,1.164437,fertiggestellte Wohngebäude
2,Hamburg,7.976157,8.242170,9.143216,7.250221,5.865467,92.023843,91.757830,90.856784,92.749779,...,38.138768,40.433100,41.136163,55.257870,1.362475,1.198861,4.425149,5.526083,3.339393,fertiggestellte Wohngebäude
3,Niedersachsen,15.535673,18.121197,21.380828,20.966418,20.774876,84.464327,81.878803,78.619172,79.033582,...,6.693340,6.032300,7.654717,9.194906,0.507398,0.275069,0.514801,0.758435,0.259011,fertiggestellte Wohngebäude
4,Bremen,4.927353,8.592058,2.326870,7.053717,6.030856,95.072647,91.407942,97.673130,92.946283,...,28.447653,30.138504,37.601736,40.603086,0.126342,0.144404,0.886427,1.519262,0.841515,fertiggestellte Wohngebäude
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11008008,Berlin-Neukölln,NaN,NaN,5.533063,3.609626,3.000000,NaN,NaN,94.466937,96.390374,...,NaN,37.651822,74.866310,90.400000,NaN,NaN,0.000000,0.000000,0.000000,fertiggestellte Wohngebäude
11009009,Berlin-Treptow-Köpenick,NaN,NaN,5.057471,6.794294,17.111995,NaN,NaN,94.942529,93.205706,...,NaN,52.321839,66.028529,44.229676,NaN,NaN,0.000000,3.003003,0.284252,fertiggestellte Wohngebäude
11010010,Berlin-Marzahn-Hellersdorf,NaN,NaN,15.795455,6.398349,9.476584,NaN,NaN,84.204545,93.601651,...,NaN,45.795455,65.428277,81.157025,NaN,NaN,0.340909,0.000000,0.000000,fertiggestellte Wohngebäude


In [172]:
#save dataframe to file
piv_df.to_csv("data/final_data_landkreis_newbuildings_renewables.csv")

## Static Aggregated Energy categories

In [173]:
agg_pivot = pivot.copy()
agg_pivot

energy,year,Biogas/Biomethan,Fernwärme/Fernkälte,Gas,Geothermie,Holz,Keine Energie (einschl. Passivhaus),Solarthermie,Sonstige Biomasse,Sonstige Heizenergie,Strom,Umweltthermie (Luft/Wasser),Öl,ort_ags,ort_name,renewables,fossils
0,2016,0.414990,19.123869,49.870626,5.717292,5.278820,0.055543,0.494014,0.512529,0.411378,0.591553,16.798298,0.731087,0,Deutschland,29.271487,70.728513
1,2017,0.448011,19.228634,46.271455,5.482344,5.773879,0.058132,0.492594,0.503521,0.358846,0.543295,20.113729,0.725559,0,Deutschland,32.872210,67.127790
2,2018,0.290789,21.294326,41.498508,5.846834,6.000411,0.080565,0.624378,0.347856,0.707461,0.658367,22.029901,0.620602,0,Deutschland,35.220735,64.779265
3,2019,0.391204,22.645564,40.049073,6.003339,5.103123,0.070685,0.470014,0.348143,0.549230,0.747878,23.011582,0.610165,0,Deutschland,35.398090,64.601910
4,2020,0.422788,23.097073,37.808091,5.885327,4.605475,0.049402,0.412448,0.415128,0.398661,0.928295,25.459552,0.517762,0,Deutschland,37.250119,62.749881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2016,0.000000,2.970297,76.237624,5.940594,0.000000,0.000000,0.990099,0.990099,0.000000,0.000000,12.871287,0.000000,16077,"Altenburger Land, Kreis",20.792079,79.207921
1,2017,0.000000,4.411765,45.588235,8.823529,5.882353,0.000000,2.941176,1.470588,0.000000,0.000000,30.882353,0.000000,16077,"Altenburger Land, Kreis",50.000000,50.000000
2,2018,0.000000,6.944444,36.111111,12.500000,2.777778,0.000000,0.000000,0.000000,0.000000,8.333333,33.333333,0.000000,16077,"Altenburger Land, Kreis",48.611111,51.388889
3,2019,0.000000,69.620253,15.189873,1.898734,0.632911,0.000000,0.632911,0.000000,0.000000,0.632911,10.759494,0.632911,16077,"Altenburger Land, Kreis",13.924051,86.075949


In [174]:
agg_pivot['andere erneuerbare Energien'] = agg_pivot[["Solarthermie","Geothermie","Holz","Biogas/Biomethan","Sonstige Biomasse","Keine Energie (einschl. Passivhaus)"]].sum(axis=1)
agg_pivot['andere fossile Energie'] = agg_pivot[["Gas","Öl","Fernwärme/Fernkälte","Sonstige Heizenergie"]].sum(axis=1)
agg_pivot

energy,year,Biogas/Biomethan,Fernwärme/Fernkälte,Gas,Geothermie,Holz,Keine Energie (einschl. Passivhaus),Solarthermie,Sonstige Biomasse,Sonstige Heizenergie,Strom,Umweltthermie (Luft/Wasser),Öl,ort_ags,ort_name,renewables,fossils,andere erneuerbare Energien,andere fossile Energie
0,2016,0.414990,19.123869,49.870626,5.717292,5.278820,0.055543,0.494014,0.512529,0.411378,0.591553,16.798298,0.731087,0,Deutschland,29.271487,70.728513,12.473188,70.136960
1,2017,0.448011,19.228634,46.271455,5.482344,5.773879,0.058132,0.492594,0.503521,0.358846,0.543295,20.113729,0.725559,0,Deutschland,32.872210,67.127790,12.758481,66.584495
2,2018,0.290789,21.294326,41.498508,5.846834,6.000411,0.080565,0.624378,0.347856,0.707461,0.658367,22.029901,0.620602,0,Deutschland,35.220735,64.779265,13.190834,64.120898
3,2019,0.391204,22.645564,40.049073,6.003339,5.103123,0.070685,0.470014,0.348143,0.549230,0.747878,23.011582,0.610165,0,Deutschland,35.398090,64.601910,12.386508,63.854032
4,2020,0.422788,23.097073,37.808091,5.885327,4.605475,0.049402,0.412448,0.415128,0.398661,0.928295,25.459552,0.517762,0,Deutschland,37.250119,62.749881,11.790567,61.821587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2016,0.000000,2.970297,76.237624,5.940594,0.000000,0.000000,0.990099,0.990099,0.000000,0.000000,12.871287,0.000000,16077,"Altenburger Land, Kreis",20.792079,79.207921,7.920792,79.207921
1,2017,0.000000,4.411765,45.588235,8.823529,5.882353,0.000000,2.941176,1.470588,0.000000,0.000000,30.882353,0.000000,16077,"Altenburger Land, Kreis",50.000000,50.000000,19.117647,50.000000
2,2018,0.000000,6.944444,36.111111,12.500000,2.777778,0.000000,0.000000,0.000000,0.000000,8.333333,33.333333,0.000000,16077,"Altenburger Land, Kreis",48.611111,51.388889,15.277778,43.055556
3,2019,0.000000,69.620253,15.189873,1.898734,0.632911,0.000000,0.632911,0.000000,0.000000,0.632911,10.759494,0.632911,16077,"Altenburger Land, Kreis",13.924051,86.075949,3.164557,85.443038


In [175]:
agg_long = agg_pivot[['year','ort_ags','ort_name','Umweltthermie (Luft/Wasser)','andere erneuerbare Energien','Gas','andere fossile Energie','Strom','renewables','fossils']].copy()
agg_long.columns = ['Jahr','AGS','Name','Umweltthermie','andere erneuerbare Energien','Gas','andere fossile Energie','Strom','renewables','fossils']

#change year column to string to facilitate renaming columns after second pivot
agg_long['Jahr'] = agg_long['Jahr'].astype(str)
agg_long

,Jahr,AGS,Name,Umweltthermie,andere erneuerbare Energien,Gas,andere fossile Energie,Strom,renewables,fossils
0,2016,0,Deutschland,16.798298,12.473188,49.870626,70.136960,0.591553,29.271487,70.728513
1,2017,0,Deutschland,20.113729,12.758481,46.271455,66.584495,0.543295,32.872210,67.127790
2,2018,0,Deutschland,22.029901,13.190834,41.498508,64.120898,0.658367,35.220735,64.779265
3,2019,0,Deutschland,23.011582,12.386508,40.049073,63.854032,0.747878,35.398090,64.601910
4,2020,0,Deutschland,25.459552,11.790567,37.808091,61.821587,0.928295,37.250119,62.749881
...,...,...,...,...,...,...,...,...,...,...
0,2016,16077,"Altenburger Land, Kreis",12.871287,7.920792,76.237624,79.207921,0.000000,20.792079,79.207921
1,2017,16077,"Altenburger Land, Kreis",30.882353,19.117647,45.588235,50.000000,0.000000,50.000000,50.000000
2,2018,16077,"Altenburger Land, Kreis",33.333333,15.277778,36.111111,43.055556,8.333333,48.611111,51.388889
3,2019,16077,"Altenburger Land, Kreis",10.759494,3.164557,15.189873,85.443038,0.632911,13.924051,86.075949


In [176]:
#pivot dataframe to have both energy types and years as columns
agg_wide = agg_long.pivot(index=['AGS','Name'],columns='Jahr',values=['Umweltthermie','andere erneuerbare Energien','Gas','andere fossile Energie','Strom','renewables','fossils'])
agg_wide

Umweltthermie                        \
Jahr                                               2016       2017       2018   
AGS      Name                                                                   
0        Deutschland                          16.798298  20.113729  22.029901   
1          Schleswig-Holstein                  7.427885  11.190120  10.464687   
2          Hamburg                             2.952030   4.765473   3.473167   
3          Niedersachsen                       8.157100  11.373677  14.044920   
4          Bremen                              1.200253   6.642599   1.883657   
...                                                 ...        ...        ...   
11008008       Berlin-Neukölln                      NaN        NaN   3.778677   
11009009       Berlin-Treptow-Köpenick              NaN        NaN   3.540230   
11010010       Berlin-Marzahn-Hellersdorf           NaN        NaN  12.159091   
11011011       Berlin-Lichtenberg                   NaN        NaN   3.005303   
11012012       Berlin-Reinickendorf                 NaN        NaN  22.137405   

                                                                 \
Jahr                                            2019       2020   
AGS      Name                                                     
0        Deutschland                       23.011582  25.459552   
1          Schleswig-Holstein              11.531785  13.509048   
2          Hamburg                          4.365606   4.143144   
3          Niedersachsen                   13.100590  14.090942   
4          Bremen                           6.673901   2.314165   
...                                              ...        ...   
11008008       Berlin-Neukölln              3.208556   2.400000   
11009009       Berlin-Treptow-Köpenick      3.566066   6.822058   
11010010       Berlin-Marzahn-Hellersdorf   5.366357   8.650138   
11011011       Berlin-Lichtenberg           2.220988   1.323829   
11012012       Berlin-Reinickendorf         7.213115   5.668934   

                                          andere erneuerbare Energien  \
Jahr                                                             2016   
AGS      Name                                                           
0        Deutschland                                        12.473188   
1          Schleswig-Holstein                                9.887821   
2          Hamburg                                           5.024127   
3          Niedersachsen                                     7.378573   
4          Bremen                                            3.727100   
...                                                               ...   
11008008       Berlin-Neukölln                                    NaN   
11009009       Berlin-Treptow-Köpenick                            NaN   
11010010       Berlin-Marzahn-Hellersdorf                         NaN   
11011011       Berlin-Lichtenberg                                 NaN   
11012012       Berlin-Reinickendorf                               NaN   

                                                                            \
Jahr                                            2017       2018       2019   
AGS      Name                                                                
0        Deutschland                       12.758481  13.190834  12.386508   
1          Schleswig-Holstein              13.547904  12.760465  12.242705   
2          Hamburg                          3.476697   5.670049   2.884615   
3          Niedersachsen                    6.747520   7.335907   7.865827   
4          Bremen                           1.949458   0.443213   0.379816   
...                                              ...        ...        ...   
11008008       Berlin-Neukölln                   NaN   1.754386   0.401070   
11009009       Berlin-Treptow-Köpenick           NaN   1.517241   3.228228   
11010010       Berlin-Marzahn-Hellersdorf        NaN   3.636364   1.031992   
11011011 

In [177]:
#collapse levels of column names and join energy type to year
agg_wide.columns = agg_wide.columns.map('_'.join)

#prepend all columns with indicator name
agg_wide=agg_wide.add_prefix('buildings_newbuilding_energy_')

#reset index to recreate AGS and Name columns
agg_wide.reset_index(inplace=True)

#rename Name and AGS columns and strip whitespace
agg_wide.rename(columns={'ort_name':'Name','ort_ags':'AGS'},inplace=True)
agg_wide['Name']=agg_wide['Name'].str.strip()

#set AGS as index
agg_wide.set_index('AGS',inplace=True)

#add column for human-readable unit of measure
agg_wide['Unit'] = "fertiggestellte Wohngebäude"

agg_wide

,Name,buildings_newbuilding_energy_Umweltthermie_2016,buildings_newbuilding_energy_Umweltthermie_2017,buildings_newbuilding_energy_Umweltthermie_2018,buildings_newbuilding_energy_Umweltthermie_2019,buildings_newbuilding_energy_Umweltthermie_2020,buildings_newbuilding_energy_andere erneuerbare Energien_2016,buildings_newbuilding_energy_andere erneuerbare Energien_2017,buildings_newbuilding_energy_andere erneuerbare Energien_2018,buildings_newbuilding_energy_andere erneuerbare Energien_2019,...,buildings_newbuilding_energy_renewables_2017,buildings_newbuilding_energy_renewables_2018,buildings_newbuilding_energy_renewables_2019,buildings_newbuilding_energy_renewables_2020,buildings_newbuilding_energy_fossils_2016,buildings_newbuilding_energy_fossils_2017,buildings_newbuilding_energy_fossils_2018,buildings_newbuilding_energy_fossils_2019,buildings_newbuilding_energy_fossils_2020,Unit
AGS,,,,,,,,,,,,,,,,,,,,,
0,Deutschland,16.798298,20.113729,22.029901,23.011582,25.459552,12.473188,12.758481,13.190834,12.386508,...,32.872210,35.220735,35.398090,37.250119,70.728513,67.127790,64.779265,64.601910,62.749881,fertiggestellte Wohngebäude
1,Schleswig-Holstein,7.427885,11.190120,10.464687,11.531785,13.509048,9.887821,13.547904,12.760465,12.242705,...,24.738024,23.225152,23.774490,26.782061,82.684295,75.261976,76.774848,76.225510,73.217939,fertiggestellte Wohngebäude
2,Hamburg,2.952030,4.765473,3.473167,4.365606,4.143144,5.024127,3.476697,5.670049,2.884615,...,8.242170,9.143216,7.250221,5.865467,92.023843,91.757830,90.856784,92.749779,94.134533,fertiggestellte Wohngebäude
3,Niedersachsen,8.157100,11.373677,14.044920,13.100590,14.090942,7.378573,6.747520,7.335907,7.865827,...,18.121197,21.380828,20.966418,20.774876,84.464327,81.878803,78.619172,79.033582,79.225124,fertiggestellte Wohngebäude
4,Bremen,1.200253,6.642599,1.883657,6.673901,2.314165,3.727100,1.949458,0.443213,0.379816,...,8.592058,2.326870,7.053717,6.030856,95.072647,91.407942,97.673130,92.946283,93.969144,fertiggestellte Wohngebäude
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11008008,Berlin-Neukölln,NaN,NaN,3.778677,3.208556,2.400000,NaN,NaN,1.754386,0.401070,...,NaN,5.533063,3.609626,3.000000,NaN,NaN,94.466937,96.390374,97.000000,fertiggestellte Wohngebäude
11009009,Berlin-Treptow-Köpenick,NaN,NaN,3.540230,3.566066,6.822058,NaN,NaN,1.517241,3.228228,...,NaN,5.057471,6.794294,17.111995,NaN,NaN,94.942529,93.205706,82.888005,fertiggestellte Wohngebäude
11010010,Berlin-Marzahn-Hellersdorf,NaN,NaN,12.159091,5.366357,8.650138,NaN,NaN,3.636364,1.031992,...,NaN,15.795455,6.398349,9.476584,NaN,NaN,84.204545,93.601651,90.523416,fertiggestellte Wohngebäude


In [178]:
#save dataframe to file
agg_wide.to_csv("data/final_data_aggregated_landkreis_newbuildings_renewables.csv")

## Dynamic Aggregated Data

In [179]:
renewables = ["Solarthermie","Geothermie","Umweltthermie (Luft/Wasser)","Holz","Biogas/Biomethan","Sonstige Biomasse","Keine Energie (einschl. Passivhaus)"]
fossils = ["Gas","Strom","Öl","Fernwärme/Fernkälte","Sonstige Heizenergie"]


def agg_below_five(df,energies,name):
    temp = df.copy()
    below_five = []
    for energy in energies:
        #get all energies below 5
        if all(temp[energy] < 5):
            below_five.append(energy)

    #aggregate them
    temp[name]=bremen[below_five].sum(axis=1)
    #set originals to nan
    temp[below_five] = np.nan
    return temp

#iterate over all orte
orte = pivot.ort_ags.unique()
dyn_pre = pivot.copy()
dyn_agg = pd.DataFrame()

for ort in orte:

    snippet = dyn_pre[dyn_pre.ort_ags == ort]

    #aggregate renewables
    snippet = agg_below_five(snippet,renewables,"andere erneuerbare Energien")
    #aggregate fossils
    snippet = agg_below_five(snippet,fossils,"andere fossile Energien")
    
    dyn_agg = pd.concat([dyn_agg, snippet])

dyn_agg


energy,year,Biogas/Biomethan,Fernwärme/Fernkälte,Gas,Geothermie,Holz,Keine Energie (einschl. Passivhaus),Solarthermie,Sonstige Biomasse,Sonstige Heizenergie,Strom,Umweltthermie (Luft/Wasser),Öl,ort_ags,ort_name,renewables,fossils,andere erneuerbare Energien,andere fossile Energien
0,2016,NaN,19.123869,49.870626,5.717292,5.278820,NaN,NaN,NaN,NaN,NaN,16.798298,NaN,0,Deutschland,29.271487,70.728513,0.000000,0.252685
1,2017,NaN,19.228634,46.271455,5.482344,5.773879,NaN,NaN,NaN,NaN,NaN,20.113729,NaN,0,Deutschland,32.872210,67.127790,0.216606,2.310469
2,2018,NaN,21.294326,41.498508,5.846834,6.000411,NaN,NaN,NaN,NaN,NaN,22.029901,NaN,0,Deutschland,35.220735,64.779265,0.055402,1.329640
3,2019,NaN,22.645564,40.049073,6.003339,5.103123,NaN,NaN,NaN,NaN,NaN,23.011582,NaN,0,Deutschland,35.398090,64.601910,0.054259,1.682040
4,2020,NaN,23.097073,37.808091,5.885327,4.605475,NaN,NaN,NaN,NaN,NaN,25.459552,NaN,0,Deutschland,37.250119,62.749881,0.000000,1.823282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2016,NaN,2.970297,76.237624,5.940594,0.000000,NaN,NaN,NaN,NaN,0.000000,12.871287,NaN,16077,"Altenburger Land, Kreis",20.792079,79.207921,0.000000,0.189514
1,2017,NaN,4.411765,45.588235,8.823529,5.882353,NaN,NaN,NaN,NaN,0.000000,30.882353,NaN,16077,"Altenburger Land, Kreis",50.000000,50.000000,0.216606,2.310469
2,2018,NaN,6.944444,36.111111,12.500000,2.777778,NaN,NaN,NaN,NaN,8.333333,33.333333,NaN,16077,"Altenburger Land, Kreis",48.611111,51.388889,0.055402,0.941828
3,2019,NaN,69.620253,15.189873,1.898734,0.632911,NaN,NaN,NaN,NaN,0.632911,10.759494,NaN,16077,"Altenburger Land, Kreis",13.924051,86.075949,0.054259,1.519262


In [180]:
#create final data format
final_dyn_agg = final_data_transform(dyn_agg)


,Name,buildings_newbuilding_energy_renewables_2016,buildings_newbuilding_energy_renewables_2017,buildings_newbuilding_energy_renewables_2018,buildings_newbuilding_energy_renewables_2019,buildings_newbuilding_energy_renewables_2020,buildings_newbuilding_energy_fossils_2016,buildings_newbuilding_energy_fossils_2017,buildings_newbuilding_energy_fossils_2018,buildings_newbuilding_energy_fossils_2019,...,buildings_newbuilding_energy_Fernwärme/Fernkälte_2017,buildings_newbuilding_energy_Fernwärme/Fernkälte_2018,buildings_newbuilding_energy_Fernwärme/Fernkälte_2019,buildings_newbuilding_energy_Fernwärme/Fernkälte_2020,buildings_newbuilding_energy_Sonstige Heizenergie_2016,buildings_newbuilding_energy_Sonstige Heizenergie_2017,buildings_newbuilding_energy_Sonstige Heizenergie_2018,buildings_newbuilding_energy_Sonstige Heizenergie_2019,buildings_newbuilding_energy_Sonstige Heizenergie_2020,Unit
AGS,,,,,,,,,,,,,,,,,,,,,
0,Deutschland,29.271487,32.872210,35.220735,35.398090,37.250119,70.728513,67.127790,64.779265,64.601910,...,19.228634,21.294326,22.645564,23.097073,NaN,NaN,NaN,NaN,NaN,fertiggestellte Wohngebäude
1,Schleswig-Holstein,17.315705,24.738024,23.225152,23.774490,26.782061,82.684295,75.261976,76.774848,76.225510,...,16.598054,20.200996,23.501695,24.020456,NaN,NaN,NaN,NaN,NaN,fertiggestellte Wohngebäude
2,Hamburg,7.976157,8.242170,9.143216,7.250221,5.865467,92.023843,91.757830,90.856784,92.749779,...,38.138768,40.433100,41.136163,55.257870,1.362475,1.198861,4.425149,5.526083,3.339393,fertiggestellte Wohngebäude
3,Niedersachsen,15.535673,18.121197,21.380828,20.966418,20.774876,84.464327,81.878803,78.619172,79.033582,...,6.693340,6.032300,7.654717,9.194906,NaN,NaN,NaN,NaN,NaN,fertiggestellte Wohngebäude
4,Bremen,4.927353,8.592058,2.326870,7.053717,6.030856,95.072647,91.407942,97.673130,92.946283,...,28.447653,30.138504,37.601736,40.603086,NaN,NaN,NaN,NaN,NaN,fertiggestellte Wohngebäude
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11008008,Berlin-Neukölln,NaN,NaN,5.533063,3.609626,3.000000,NaN,NaN,94.466937,96.390374,...,NaN,37.651822,74.866310,90.400000,NaN,NaN,NaN,NaN,NaN,fertiggestellte Wohngebäude
11009009,Berlin-Treptow-Köpenick,NaN,NaN,5.057471,6.794294,17.111995,NaN,NaN,94.942529,93.205706,...,NaN,52.321839,66.028529,44.229676,NaN,NaN,NaN,NaN,NaN,fertiggestellte Wohngebäude
11010010,Berlin-Marzahn-Hellersdorf,NaN,NaN,15.795455,6.398349,9.476584,NaN,NaN,84.204545,93.601651,...,NaN,45.795455,65.428277,81.157025,NaN,NaN,NaN,NaN,NaN,fertiggestellte Wohngebäude


## Number of Buildings

In [193]:
#helper rounding function
#https://stackoverflow.com/questions/31818050/round-number-to-nearest-integer
def proper_round(num, dec=0):
    num = str(num)[:str(num).index('.')+dec+2]
    if num[-1]>='5':
        return float(num[:-2-(not dec)]+str(int(num[-2-(not dec)])+1))
    return float(num[:-1])

In [194]:
#create absolute number od buildings
# create_pivot_tables_
#         viz : df dataframe slice of your data, for example a specific landkreis
#        asProportion : bool, optional
#            if true, the visualisation will use proportion. otherwise, the total number is used. default is true
#        useWohnungen : bool, optional
#            if true, flats will be counted. otherwise, buildings will be counted (which maybe have multiple flats in them). default is true
abs_buildings = create_pivot_tables(nb_kreise,False,True)
abs_buildings

#iterate over all orte
orte = abs_buildings.ort_ags.unique()

energies = renewables + fossils

for ort in orte:
    snippet = abs_buildings[abs_buildings.ort_ags == ort]
    number_buildings = snippet[energies].sum(axis=1).mean()


    final_dyn_agg.loc[ort,"buildings_newbuilding_energy_number_buildings"] = proper_round(number_buildings)
    

final_dyn_agg



,Name,buildings_newbuilding_energy_renewables_2016,buildings_newbuilding_energy_renewables_2017,buildings_newbuilding_energy_renewables_2018,buildings_newbuilding_energy_renewables_2019,buildings_newbuilding_energy_renewables_2020,buildings_newbuilding_energy_fossils_2016,buildings_newbuilding_energy_fossils_2017,buildings_newbuilding_energy_fossils_2018,buildings_newbuilding_energy_fossils_2019,...,buildings_newbuilding_energy_Fernwärme/Fernkälte_2018,buildings_newbuilding_energy_Fernwärme/Fernkälte_2019,buildings_newbuilding_energy_Fernwärme/Fernkälte_2020,buildings_newbuilding_energy_Sonstige Heizenergie_2016,buildings_newbuilding_energy_Sonstige Heizenergie_2017,buildings_newbuilding_energy_Sonstige Heizenergie_2018,buildings_newbuilding_energy_Sonstige Heizenergie_2019,buildings_newbuilding_energy_Sonstige Heizenergie_2020,Unit,buildings_newbuilding_energy_number_buildings
AGS,,,,,,,,,,,,,,,,,,,,,
0,Deutschland,29.271487,32.872210,35.220735,35.398090,37.250119,70.728513,67.127790,64.779265,64.601910,...,21.294326,22.645564,23.097073,NaN,NaN,NaN,NaN,NaN,fertiggestellte Wohngebäude,239169.0
1,Schleswig-Holstein,17.315705,24.738024,23.225152,23.774490,26.782061,82.684295,75.261976,76.774848,76.225510,...,20.200996,23.501695,24.020456,NaN,NaN,NaN,NaN,NaN,fertiggestellte Wohngebäude,11764.0
2,Hamburg,7.976157,8.242170,9.143216,7.250221,5.865467,92.023843,91.757830,90.856784,92.749779,...,40.433100,41.136163,55.257870,1.362475,1.198861,4.425149,5.526083,3.339393,fertiggestellte Wohngebäude,8555.0
3,Niedersachsen,15.535673,18.121197,21.380828,20.966418,20.774876,84.464327,81.878803,78.619172,79.033582,...,6.032300,7.654717,9.194906,NaN,NaN,NaN,NaN,NaN,fertiggestellte Wohngebäude,25455.0
4,Bremen,4.927353,8.592058,2.326870,7.053717,6.030856,95.072647,91.407942,97.673130,92.946283,...,30.138504,37.601736,40.603086,NaN,NaN,NaN,NaN,NaN,fertiggestellte Wohngebäude,1608.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
15364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
15367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [195]:
#save dataframe to file
final_dyn_agg.to_csv("data/final_data_dynamic_aggregated_landkreis_newbuildings_renewables.csv")
final_dyn_agg

,Name,buildings_newbuilding_energy_renewables_2016,buildings_newbuilding_energy_renewables_2017,buildings_newbuilding_energy_renewables_2018,buildings_newbuilding_energy_renewables_2019,buildings_newbuilding_energy_renewables_2020,buildings_newbuilding_energy_fossils_2016,buildings_newbuilding_energy_fossils_2017,buildings_newbuilding_energy_fossils_2018,buildings_newbuilding_energy_fossils_2019,...,buildings_newbuilding_energy_Fernwärme/Fernkälte_2018,buildings_newbuilding_energy_Fernwärme/Fernkälte_2019,buildings_newbuilding_energy_Fernwärme/Fernkälte_2020,buildings_newbuilding_energy_Sonstige Heizenergie_2016,buildings_newbuilding_energy_Sonstige Heizenergie_2017,buildings_newbuilding_energy_Sonstige Heizenergie_2018,buildings_newbuilding_energy_Sonstige Heizenergie_2019,buildings_newbuilding_energy_Sonstige Heizenergie_2020,Unit,buildings_newbuilding_energy_number_buildings
AGS,,,,,,,,,,,,,,,,,,,,,
0,Deutschland,29.271487,32.872210,35.220735,35.398090,37.250119,70.728513,67.127790,64.779265,64.601910,...,21.294326,22.645564,23.097073,NaN,NaN,NaN,NaN,NaN,fertiggestellte Wohngebäude,239169.0
1,Schleswig-Holstein,17.315705,24.738024,23.225152,23.774490,26.782061,82.684295,75.261976,76.774848,76.225510,...,20.200996,23.501695,24.020456,NaN,NaN,NaN,NaN,NaN,fertiggestellte Wohngebäude,11764.0
2,Hamburg,7.976157,8.242170,9.143216,7.250221,5.865467,92.023843,91.757830,90.856784,92.749779,...,40.433100,41.136163,55.257870,1.362475,1.198861,4.425149,5.526083,3.339393,fertiggestellte Wohngebäude,8555.0
3,Niedersachsen,15.535673,18.121197,21.380828,20.966418,20.774876,84.464327,81.878803,78.619172,79.033582,...,6.032300,7.654717,9.194906,NaN,NaN,NaN,NaN,NaN,fertiggestellte Wohngebäude,25455.0
4,Bremen,4.927353,8.592058,2.326870,7.053717,6.030856,95.072647,91.407942,97.673130,92.946283,...,30.138504,37.601736,40.603086,NaN,NaN,NaN,NaN,NaN,fertiggestellte Wohngebäude,1608.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
15364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
15367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
